In [1]:
import os
import numpy as np
import cv2
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
import random
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

from google.colab import files

# Upload multiple files
uploaded = files.upload()

# Iterate through the uploaded files and process them
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes.')
    # You can add your own file processing logic here


Mounted at /content/gdrive


Saving Test.zip to Test.zip
Saving Training.zip to Training.zip
User uploaded file "Test.zip" with length 102053473 bytes.
User uploaded file "Training.zip" with length 310662552 bytes.


In [2]:
#Unzip RAR
!unzip "Training.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Training/Tomato 2/r2_280_100.jpg  
  inflating: Training/Tomato 2/r2_281_100.jpg  
  inflating: Training/Tomato 2/r2_282_100.jpg  
  inflating: Training/Tomato 2/r2_283_100.jpg  
  inflating: Training/Tomato 2/r2_284_100.jpg  
  inflating: Training/Tomato 2/r2_285_100.jpg  
  inflating: Training/Tomato 2/r2_286_100.jpg  
  inflating: Training/Tomato 2/r2_287_100.jpg  
  inflating: Training/Tomato 2/r2_289_100.jpg  
  inflating: Training/Tomato 2/r2_29_100.jpg  
  inflating: Training/Tomato 2/r2_290_100.jpg  
  inflating: Training/Tomato 2/r2_293_100.jpg  
  inflating: Training/Tomato 2/r2_294_100.jpg  
  inflating: Training/Tomato 2/r2_295_100.jpg  
  inflating: Training/Tomato 2/r2_296_100.jpg  
  inflating: Training/Tomato 2/r2_298_100.jpg  
  inflating: Training/Tomato 2/r2_299_100.jpg  
  inflating: Training/Tomato 2/r2_30_100.jpg  
  inflating: Training/Tomato 2/r2_301_100.jpg  
  inflating: Training/Tomato 2/r2_31_10

In [3]:
#Unzip RAR
!unzip "Test.zip"

Streaming output truncated to the last 5000 lines.
  inflating: Test/Plum 3/r_156_100.jpg  
  inflating: Test/Plum 3/r_158_100.jpg  
  inflating: Test/Plum 3/r_16_100.jpg  
  inflating: Test/Plum 3/r_160_100.jpg  
  inflating: Test/Plum 3/r_163_100.jpg  
  inflating: Test/Plum 3/r_164_100.jpg  
  inflating: Test/Plum 3/r_167_100.jpg  
  inflating: Test/Plum 3/r_169_100.jpg  
  inflating: Test/Plum 3/r_17_100.jpg  
  inflating: Test/Plum 3/r_171_100.jpg  
  inflating: Test/Plum 3/r_172_100.jpg  
  inflating: Test/Plum 3/r_174_100.jpg  
  inflating: Test/Plum 3/r_175_100.jpg  
  inflating: Test/Plum 3/r_176_100.jpg  
  inflating: Test/Plum 3/r_177_100.jpg  
  inflating: Test/Plum 3/r_178_100.jpg  
  inflating: Test/Plum 3/r_179_100.jpg  
  inflating: Test/Plum 3/r_180_100.jpg  
  inflating: Test/Plum 3/r_181_100.jpg  
  inflating: Test/Plum 3/r_182_100.jpg  
  inflating: Test/Plum 3/r_183_100.jpg  
  inflating: Test/Plum 3/r_184_100.jpg  
  inflating: Test/Plum 3/r_185_100.jpg  
  inflat

In [2]:
# Define the image size
img_size = 80  # VGG16 expects 224x224 images

# Function to load and process images
def load_and_process_images(directory, classes, img_size):
    all_arrays = []
    for class_num, fruit_class in enumerate(classes):
        path = os.path.join(directory, fruit_class)
        for img_name in os.listdir(path):
            try:
                img_path = os.path.join(path, img_name)
                img_array = cv2.imread(img_path)
                if img_array is not None:
                    img_array = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
                    img_array = cv2.resize(img_array, (img_size, img_size))
                    all_arrays.append([img_array, class_num])
            except Exception as e:
                print(f"Error loading image {img_name}: {e}")
    return np.array(all_arrays, dtype=object)


In [3]:
np.random.seed(1234)

# Directories and classes
train_directory = "/content/Training"
test_directory = "/content/Test"
classes = ['Apple Braeburn', 'Apple Crimson Snow', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Pink Lady', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3',
            'Apple Red Delicious', 'Apple Red Yellow 1', 'Apple Red Yellow 2', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Lady Finger', 'Banana Red', 'Beetroot', 'Blueberry', 'Cactus fruit',
            'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cauliflower', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Cherry Wax Black', 'Cherry Wax Red', 'Cherry Wax Yellow', 'Chestnut', 'Clementine', 'Cocos',
            'Corn', 'Corn Husk', 'Cucumber Ripe', 'Cucumber Ripe 2', 'Dates', 'Eggplant', 'Fig', 'Ginger Root', 'Granadilla']

In [4]:
# Load and process training images
train_data = load_and_process_images(train_directory, classes, img_size)
print(f"Loaded {len(train_data)} images from {len(classes)} classes for training.")

Loaded 21915 images from 44 classes for training.


In [5]:
# Load and process test images
test_data = load_and_process_images(test_directory, classes, img_size)
print(f"Loaded {len(test_data)} images from {len(classes)} classes for testing.")

Loaded 7333 images from 44 classes for testing.


In [6]:
# Shuffle and separate training data
np.random.shuffle(train_data)
X_train = []
Y_train = []
for features, label in train_data:
    X_train.append(features)
    Y_train.append(label)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [7]:
# Shuffle and separate test data
np.random.shuffle(test_data)
X_test = []
Y_test = []
for features, label in test_data:
    X_test.append(features)
    Y_test.append(label)
X_test = np.array(X_test)
Y_test = np.array(Y_test)

In [8]:
X_train = X_train / 255.0
X_test = X_test / 255.0

print("shape of X_train= ", X_train.shape)
print("shape of X_test=  ", X_test.shape)

shape of X_train=  (21915, 80, 80, 3)
shape of X_test=   (7333, 80, 80, 3)


In [9]:
# One-hot encoding of labels
num_classes = 44  # Adjust this based on the number of classes in your dataset
Y_train = to_categorical(Y_train, num_classes=num_classes)
Y_test = to_categorical(Y_test, num_classes=num_classes)

In [10]:
# Split the dataset
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=40)

In [11]:
# Load VGG16 model + higher level layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [12]:
# Add custom layers on top of VGG16 base
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [14]:
# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of VGG16 so they are not trained
for layer in base_model.layers:
    layer.trainable = False

# Defining optimizer
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

# Compile the model
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

epochs = 10
batch_size = 10

In [15]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,  # Increased for more augmentation
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False
)

# Fitting the model
datagen.fit(x_train)
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    epochs=epochs,
    validation_data=(x_val, y_val),
    steps_per_epoch=x_train.shape[0] // batch_size,
    callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True), ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')]
)

Epoch 1/10
1753/1753 [==============================] - ETA: 0s - loss: 1.3863 - accuracy: 0.6010

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1753/1753 [==============================] - 1544s 879ms/step - loss: 1.3863 - accuracy: 0.6010 - val_loss: 0.4555 - val_accuracy: 0.8560
Epoch 2/10
1753/1753 [==============================] - 1516s 865ms/step - loss: 0.6127 - accuracy: 0.8034 - val_loss: 0.3581 - val_accuracy: 0.8811
Epoch 3/10
1753/1753 [==============================] - 1527s 871ms/step - loss: 0.4634 - accuracy: 0.8441 - val_loss: 0.2455 - val_accuracy: 0.9204
Epoch 4/10
1753/1753 [==============================] - 1513s 863ms/step - loss: 0.4046 - accuracy: 0.8638 - val_loss: 0.2075 - val_accuracy: 0.9322
Epoch 5/10
1753/1753 [==============================] - 1517s 866ms/step - loss: 0.3663 - accuracy: 0.8738 - val_loss: 0.1899 - val_accuracy: 0.9343
Epoch 6/10
1753/1753 [==============================] - 1505s 858ms/step - loss: 0.3377 - accuracy: 0.8845 - val_loss: 0.1478 - val_accuracy: 0.9560
Epoch 7/10
1753/1753 [==============================] - 1515s 864ms/step - loss: 0.3285 - accuracy: 0.8881 - val_loss

In [16]:
# Save the model
model.save('fruit_classifier.h5')